In [1]:
import os
from dotenv import load_dotenv
import mlflow
from mlflow.models import infer_signature
from mlflow_utils import get_mlflow_experiment


from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import pandas as pd
 
load_dotenv()

True

In [2]:
# conectar con mlflow y minio (por http)
mlflow.set_tracking_uri("http://127.0.0.1:5000")

os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://127.0.0.1:9000"
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('ACCESS_KEY')

# Inference

In [3]:
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, random_state=42)
X = pd.DataFrame(X, columns=["feature_{}".format(i) for i in range(10)])
y = pd.DataFrame(y, columns=["target"])

_, X_test, _, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

In [4]:
# load model
run_id = "8af1a1a781064167b3f8caee4bf90e00"
model_uri = f'runs:/{run_id}/random_forest_classifier'
rfc = mlflow.sklearn.load_model(model_uri=model_uri)

/home/oecorrechag/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
y_pred = rfc.predict(X_test)
y_pred = pd.DataFrame(y_pred, columns=["prediction"])
acc = accuracy_score(y_test, y_pred)
print(acc)

0.945


In [6]:
X_test.shape, y_pred.shape

((200, 10), (200, 1))

In [7]:
print('tracking uri:', mlflow.get_tracking_uri())
print('artifact uri:', mlflow.get_artifact_uri())

tracking uri: http://127.0.0.1:5000
artifact uri: s3://mlflow/0/fdee54b2eae64582a1e610849414a1e6/artifacts


In [8]:
mlflow.end_run()

In [9]:
print('ok_')

ok_
